In [1]:
import re
import time
import requests
import numpy as np
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
chrome_driver = r"C:\Users\HP\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
download_file_path = r"C:\Users\HP\Downloads\chromedriver-win64\chromedriver-win64"


prefs = {
    "download.default_directory": download_file_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
    }
options = webdriver.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--headless")

# Adjust the timeout value according to your needs
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")

In [3]:
url = 'https://www.shoppersstop.com/'
shopper = webdriver.Chrome()
shopper.get(url)

There was an error managing chrome (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json): connection error: connection reset); using driver found in the cache


In [4]:
search_box = shopper.find_element(By.NAME, "text")
search_box.clear()
search_box.send_keys("Shirts") # enter your name in the search box
sleep(1)
search_box.submit() # submit the search

In [5]:
# Find all product elements using class name
product_elements = shopper.find_elements(By.CLASS_NAME, "pro-box")
product_oldprize = shopper.find_elements(By.XPATH, '//*[@id="pdpImage"]/div[3]/div[1]/div/ul/li[2]/span')
dis_count = shopper.find_elements(By.XPATH,'//*[@id="pdpImage"]/div[3]/div[1]/div/ul/li[3]/span')
pro_prize = shopper.find_elements(By.XPATH,'//*[@id="pdpImage"]/div[3]/div[1]/div/ul/li[1]')
product_link = shopper.find_elements(By.XPATH,'//*[@id="5HashPosition"]/div')

In [6]:
# Create a list to store dictionaries representing products
products_Name = []
ProductBrand = []
category = []
productcode = []
productSize =[]
Oldprize = []
discount =[]
Prize = []

for i in range(15):
    print('Data collecting',i+1)
    for product_element in product_elements:
        # Extract data attributes
        product_name = product_element.get_attribute("data-product-name")
        products_Name.append(product_name)
    for product_element1 in product_elements:
        product_brand = product_element1.get_attribute("data-product-brandname")
        ProductBrand.append(product_brand)
    for product_element2 in product_elements:
        product_c = product_element2.get_attribute("data-category-name")
        category.append(product_c)
    for product_element3 in product_elements:
        product_code = product_element3.get_attribute("data-product-code")
        productcode.append(product_code)
    for product_element4 in product_elements:
        product_Sz = product_element4.get_attribute("data-product-sizes")
        productSize.append(product_Sz)
    for old_prize in product_oldprize:
            temp = old_prize.text
            Oldprize.append(temp)
    for Dis  in dis_count:
        temp = Dis.text
        discount.append(temp)
    for S_prize in pro_prize:
        temp = S_prize.text
        Prize.append(temp)
    shopper.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wait = WebDriverWait(shopper,60)
    load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="loadmore"]')))
    shopper.execute_script("arguments[0].scrollIntoView();", load_more_button)
    shopper.execute_script("arguments[0].click();", load_more_button)
    sleep(3)
        
shopper.quit()

Data collecting 1
Data collecting 2
Data collecting 3
Data collecting 4
Data collecting 5
Data collecting 6
Data collecting 7
Data collecting 8
Data collecting 9
Data collecting 10
Data collecting 11
Data collecting 12
Data collecting 13
Data collecting 14
Data collecting 15


In [7]:
df_overall = {
    'Product Name': products_Name,
    'BrandName': ProductBrand,
    'category': category,
    'Product Prize': Prize,
    'Discount Off': discount,
    'Product Oldprize': Oldprize,
    'Product Size': productSize,
    'Product Code': productcode
}
max_length = max(len(value) for value in df_overall.values())
df_overall = {key: value + [None] * (max_length - len(value)) for key, value in df_overall.items()}
df_overall = pd.DataFrame(df_overall)
df_overall.head()

,Product Name,BrandName,category,Product Prize,Discount Off,Product Oldprize,Product Size,Product Code
0,Stripes Cotton Slim Fit Men's Shirt - Blue,STOP,Formal Shirts,Rs 699,61% off,Rs 1799,"[39, 40, 42]",A22SHST12AGBL_BLUE
1,Checks Cotton Slim Fit Men's Casual Shirt - Pink,LIFE,Casual Shirts,Rs 749,50% off,Rs 1499,"[Large, Medium, Small, X-Large]",A23LDCHCK137PK_PINK
2,Printed Cotton Slim Fit Men's Shirt - Navy,STOP,Formal Shirts,Rs 699,53% off,Rs 1499,"[39, 40, 42, 44]",A22SHSP61SWNV_NAVY
3,Stripes Cotton Regular Fit Men's Casual Shirt ...,URI,Casual Shirts,Rs 899,55% off,Rs 1999,"[Large, Small, X-Large]",URIFS130MULTI_MULTI
4,Printed Cotton Slim Fit Mens Shirt - Wine,STOP,Formal Shirts,Rs 699,53% off,Rs 1499,"[39, 40]",S23STSTNPRT29WN_WINE


In [8]:

# Assuming df is your DataFrame
df = pd.DataFrame(df_overall)  

# Specify the Excel file path
excel_file_path = 'C:\\Users\\HP\\Downloads\\shoppers.xlsx'

# Write DataFrame to Excel
df.to_excel(excel_file_path, index=False)

print(f'DataFrame has been written to {excel_file_path}')

DataFrame has been written to C:\Users\HP\Downloads\shoppers.xlsx
